<a href="https://colab.research.google.com/github/theoem/Data-Linkage/blob/master/Data-Linkage/Record-Linking-Fuzzy-Matching-Teodoras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CSC8381 Record Linking - Teodoras Celencevicius 190364247


In [272]:
!pip install fuzzymatcher

In [273]:
!pip install recordlinkage

In [274]:
import pandas as pd
from pathlib import Path
import fuzzymatcher
import recordlinkage
from sklearn.metrics import f1_score

### Using fuzzymatcher

In [275]:
acm_data = pd.read_csv(
    'https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/ACM.csv'
)
dblp_data = pd.read_csv(
    'https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/dblp.csv'
)

In [276]:
acm_data.head()

,Unnamed: 0,id,title,authors,venue,year
0,0,304586,The WASA2 object-oriented workflow management ...,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
1,1,304587,A user-centered interface for querying distrib...,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999
2,2,304589,"World Wide Database-integrating the Web, CORBA...","Athman Bouguettaya, Boualem Benatallah, Lily H...",International Conference on Management of Data,1999
3,3,304590,XML-based information mediation with MIX,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228...",International Conference on Management of Data,1999
4,4,304582,The CCUBE constraint object-oriented database ...,"Alexander Brodsky, Victor E. Segal, Jia Chen, ...",International Conference on Management of Data,1999


In [277]:
dblp_data.head()

,Unnamed: 0,id,title,authors,venue,year
0,0,journals/sigmod/Mackay99,Semantic Integration of Environmental Models f...,D. Scott Mackay,SIGMOD Record,1999
1,1,conf/vldb/PoosalaI96,Estimation of Query-Result Distribution and it...,"Viswanath Poosala, Yannis E. Ioannidis",VLDB,1996
2,2,conf/vldb/PalpanasSCP02,Incremental Maintenance for Non-Distributive A...,"Themistoklis Palpanas, Richard Sidle, Hamid Pi...",VLDB,2002
3,3,conf/vldb/GardarinGT96,Cost-based Selection of Path Expression Proces...,"Zhao-Hui Tang, Georges Gardarin, Jean-Robert G...",VLDB,1996
4,4,conf/vldb/HoelS95,Benchmarking Spatial Join Operations with Spat...,"Erik G. Hoel, Hanan Samet",VLDB,1995


In [278]:
# Deleting any rows where author is not defined.

acm_data.dropna(subset=['authors'], inplace=True)
dblp_data.dropna(subset=['authors'], inplace=True)

In [279]:
## dataframe sizes

print("raw dataframe number of records :")
print("acm_data:  {x}".format(x=acm_data['id'].count()))
print("dblp_data:  {x}".format(x=dblp_data['id'].count()))

raw dataframe number of records :
acm_data:  2280
dblp_data:  2616


In [280]:
cond = acm_data['authors'].str.startswith('A')
print (acm_data)
acm_data.loc[cond].sort_values(by=['authors'], ascending=True)[['authors', 'title']].head(25)

      Unnamed: 0  ...  year
0              0  ...  1999
1              1  ...  1999
2              2  ...  1999
3              3  ...  1999
4              4  ...  1999
...          ...  ...   ...
2289        2289  ...  1994
2290        2290  ...  2003
2291        2291  ...  1994
2292        2292  ...  2003
2293        2293  ...  1994

[2280 rows x 6 columns]


,authors,title
1054,"A. A. Diwan, Sanjeeva Rane, S. Seshadri, S. Su...",Clustering Techniques for Minimizing External ...
1741,"A. Biliris, S. Dar, N. Gehani, H. V. Jagadish,...",ASSET: a system for supporting extended transa...
235,"A. Biliris, T. A. Funkhouser, W. O'Connell, E....",BeSS: storage support for interactive visualiz...
1565,"A. Dogac, C. Dengi, E. Kilic, G. Ozhan, F. Ozc...",METU interoperable database system
218,"A. Formica, H. D. Groger, M. Missikoff",An efficient method for checking object-orient...
1632,"A. Kemper, G. Moerkotte, K. Peithner, M. Stein...",Optimizing disjunctive queries with expensive ...
118,"A. M. Ouksel, A. Sheth",Semantic interoperability in global informatio...
2186,"A. Maedche, B. Motik, L. Stojanovic",Managing multiple and distributed ontologies o...
1529,"A. P. Buchmann, A. Deutsch, J. Zimmermann, M. ...",The REACH active OODBMS
1439,"A. Prasad Sistla, Clement T. Yu, Chengwen Liu,...",Similarity based Retrieval of Pictures Using I...


In [281]:
cond = dblp_data['authors'].str.startswith('A')
dblp_data.loc[cond].sort_values(by=['authors'], ascending=True)[['authors', 'title']].head(25)

,authors,title
207,"A. A. Diwan, S. Sudarshan, Sanjeeva Rane, S. S...",Clustering Techniques for Minimizing External ...
1279,"A. Kumaran, Jayant R. Haritsa",On the Costs of Multilingualism in Database Sy...
433,"A. Prasad Sistla, Isabel F. Cruz, Ashfaq A. Kh...",Research Activities in Database Management and...
1587,"A. Prasad Sistla, Ouri Wolfson, Yixiu Huang",Data Replication for Mobile Computers
1128,"A. Prasad Sistla, Robert H. Sloan, Yelena Yesh...",Towards a Theory of Cost Management for Digita...
1642,A. R. Dasgupta,Large Databases for Remote Sensing and GIS
703,"A. R. Hurson, Simin H. Pakzad, M. W. Bright",Automated Resolution of Semantic Heterogeneity...
2057,"Aashu Virmani, Tomasz Imielinski",DataMine - Interactive Rule Discovery System
396,"Abdelsalam Helal, Hua Li",Rainbow: Distributed Database System for Class...
2033,"Abraham Silberschatz, Christos Faloutsos, Yoss...",Modeling Skewed Distribution Using Multifracta...


In [282]:
dblp_data.columns

Index(['Unnamed: 0', 'id', 'title', 'authors', 'venue', 'year'], dtype='object')

In [283]:
# Columns to match on from df_left
left_on = ['title', 'authors', 'venue', 'year']

# Columns to match on from df_right
right_on = ['title', 'authors', 'venue', 'year']

In [284]:
## try link_table first
linked = fuzzymatcher.link_table(acm_data,
                                 dblp_data,
                                 left_on,
                                 right_on,
                                 left_id_col='id',
                                 right_id_col='id')

In [285]:
linked.head()

,__id_left,__id_right,match_score,match_rank,title_left,title_right,authors_left,authors_right,venue_left,venue_right,year_left,year_right
0,304586,conf/sigmod/VossenW99,2.559548,1,The WASA2 object-oriented workflow management ...,The WASA2 Object-Oriented Workflow Management ...,"Gottfried Vossen, Mathias Weske","Mathias Weske, Gottfried Vossen",International Conference on Management of Data,SIGMOD Conference,1999,1999
1,304587,conf/sigmod/CruzJ99,2.721471,1,A user-centered interface for querying distrib...,A User-Centered Interface for Querying Distrib...,"Isabel F. Cruz, Kimberly M. James","Kimberly M. James, Isabel F. Cruz",International Conference on Management of Data,SIGMOD Conference,1999,1999
2,304589,conf/sigmod/BouguettayaBH99,4.568259,1,"World Wide Database-integrating the Web, CORBA...","World Wide Database - Integrating the Web, COR...","Athman Bouguettaya, Boualem Benatallah, Lily H...","Kevin Smith, Athman Bouguettaya, Mourad Ouzzan...",International Conference on Management of Data,SIGMOD Conference,1999,1999
3,304590,conf/sigmod/BaruGLMPVC99,3.630595,1,XML-based information mediation with MIX,XML-Based Information Mediation with MIX,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228...","Yannis Papakonstantinou, Richard Marciano, Cha...",International Conference on Management of Data,SIGMOD Conference,1999,1999
4,304582,conf/sigmod/BrodskySCE99,2.969856,1,The CCUBE constraint object-oriented database ...,The CCUBE Constraint Object-Oriented Database ...,"Alexander Brodsky, Victor E. Segal, Jia Chen, ...","Alexander Brodsky, Pavel A. Exarkhopoulo, Vict...",International Conference on Management of Data,SIGMOD Conference,1999,1999


In [286]:
# Now perform the match
matched_results = fuzzymatcher.fuzzy_left_join(acm_data,
                                               dblp_data,
                                               left_on,
                                               right_on,
                                               left_id_col='id',
                                               right_id_col='id')

In [287]:
matched_results[["authors_left", "authors_right"]].head()

,authors_left,authors_right
0,"Gottfried Vossen, Mathias Weske","Mathias Weske, Gottfried Vossen"
1,"Isabel F. Cruz, Kimberly M. James","Kimberly M. James, Isabel F. Cruz"
2,"Athman Bouguettaya, Boualem Benatallah, Lily H...","Kevin Smith, Athman Bouguettaya, Mourad Ouzzan..."
3,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228...","Yannis Papakonstantinou, Richard Marciano, Cha..."
4,"Alexander Brodsky, Victor E. Segal, Jia Chen, ...","Alexander Brodsky, Pavel A. Exarkhopoulo, Vict..."


In [288]:
# Reorder the columns to make viewing easier
cols = [
    "best_match_score", "title_left",	"title_right", "authors_left", 
    "authors_right", "venue_left",	"venue_right", "year_left", "year_right",
    "id_left", "id_right"
]

In [325]:
# Let's see the best matches
matched_results[cols].sort_values(by=['best_match_score'], ascending=False).head(5)

,best_match_score,title_left,title_right,authors_left,authors_right,venue_left,venue_right,year_left,year_right,id_left,id_right
25271,9.284321,The Propel Distributed Services Platform,The Propel Distributed Services Platform,"Michael J. Carey, Steve Kirsch, Mary Roth, Ber...","Runping Qi, Kuan Yee, Brian Sterling, Nicolas ...",Very Large Data Bases,VLDB,2001,2001,672199,conf/vldb/CareyKRLABFLOPQRSSUVWY01
34596,9.233475,A mapping mechanism to support bitmap index an...,A mapping mechanism to support bitmap index an...,"Eugene Inseok Chong, Jagannathan Srinivasan, S...","Mahesh Jagannath, Eugene Inseok Chong, Aravind...",ACM SIGMOD Record,SIGMOD Record,2003,2003,777000,journals/sigmod/ChongFTSYKDJJ03
29317,9.165640,TSQL2 language specification,TSQL2 Language Specification,"Richard Thomas Snodgrass, Ilsoo Ahn, Gadi Aria...","Wolfgang Käfer, Curtis E. Dyreson, Arie Segev,...",ACM SIGMOD Record,SIGMOD Record,1994,1994,181562,journals/sigmod/SnodgrassAABCDEGJKKKLLRSSS94
28653,9.091846,A TSQL2 tutorial,A TSQL2 Tutorial,"Richard Thomas Snodgrass, Ilsoo Ahn, Gad Ariav...","James Clifford, Ramez Elmasri, Michael D. Soo,...",ACM SIGMOD Record,SIGMOD Record,1994,1994,187449,journals/sigmod/SnodgrassAABCDEGJKKKLLRSSS94a
427,8.605697,PowerBookmarks: a system for personalizable We...,PowerBookmarks: A System for Personalizable We...,"Wen-Syan Li, Quoc Vu, Edward Chang, Divyakant ...","Kazuyuki Shimazu, Sougata Mukherjea, Yi-Leh Wu...",International Conference on Management of Data,SIGMOD Conference,1999,1999,304578,conf/sigmod/LiVCAHMWBCHIKSS99


In [326]:
# Now the worst matches
matched_results[cols].sort_values(by=['best_match_score'],
                                  ascending=True).head(5)

,best_match_score,title_left,title_right,authors_left,authors_right,venue_left,venue_right,year_left,year_right,id_left,id_right
19672,-1.972738,David DeWitt speaks out: on rethinking the CS ...,Gio Wiederhold Speaks Out on Moving into Acade...,Marianne Winslett,Marianne Winslett,ACM SIGMOD Record,SIGMOD Record,2002,2001,565127,journals/sigmod/Winslett01a
29431,-1.682680,A consensus glossary of temporal database conc...,A TSQL2 Tutorial,"Curtis Dyreson, Fabio Grandi, Wolfgang K&#228;...","James Clifford, Ramez Elmasri, Michael D. Soo,...",ACM SIGMOD Record,SIGMOD Record,1994,1994,181560,journals/sigmod/SnodgrassAABCDEGJKKKLLRSSS94a
5674,-1.278196,Challenges in automating declarative business ...,Expressing Business Rules,Val Huber,Ronald G. Ross,International Conference on Management of Data,SIGMOD Conference,2000,2000,335455,conf/sigmod/Ross00
34067,-1.271728,Abstracts of invited industrial track presenta...,Objectivity Industrial Exhibit,Zachary Ives,Objectivity,International Conference on Management of Data,VLDB,2003,1998,872833,conf/vldb/Objectivity98
5465,-1.263943,Going beyond personalization: rule engines at ...,Delivering High Availability for Inktomi Searc...,Eric Kintzer,Eric A. Brewer,International Conference on Management of Data,SIGMOD Conference,2000,1998,335459,conf/sigmod/Brewer98


In [329]:
# Look at the matches around 1
matched_results[cols].query("best_match_score <= 1").sort_values(
    by=['best_match_score'], ascending=False).head(5)

,best_match_score,title_left,title_right,authors_left,authors_right,venue_left,venue_right,year_left,year_right,id_left,id_right
3503,0.995988,MQseries and CICS link for Lotus Notes,MQSeries and CICS Link for Lotus Notes,Lotus Development Corp.,?,ACM SIGMOD Record,SIGMOD Record,1996,1996,381900,journals/sigmod/Anonymous96
2297,0.995680,Keynote Address,Keynote Address,Robert S. Epstein,Robert S. Epstein,International Conference on Management of Data,SIGMOD Conference,1995,1995,277955,conf/sigmod/Epstein95
25441,0.990004,The Oracle Warehouse,The Oracle Warehouse,Gary Hallmark,Gary Hallmark,Very Large Data Bases,VLDB,1995,1995,673156,conf/vldb/Hallmark95
25279,0.984455,Altruistic locking,Altruistic Locking,"Kenneth Salem, H&#233;ctor Garc&#237;a-Molina,...","Jeannie Shands, Kenneth Salem, Hector Garcia-M...",ACM Transactions on Database Systems (TODS),ACM Trans. Database Syst.,1994,1994,174639,journals/tods/SalemGS94
18376,0.973914,Mid-tier caching: the TimesTen approach,Mid-tier caching: the TimesTen approach,The TimesTen Team,Times-Ten Team,International Conference on Management of Data,SIGMOD Conference,2002,2002,564761,conf/sigmod/Team02


In [292]:
cols

['best_match_score',
 'title_left',
 'title_right',
 'authors_left',
 'authors_right',
 'venue_left',
 'venue_right',
 'year_left',
 'year_right',
 'id_left',
 'id_right']

In [339]:
matched_results_final = matched_results[cols].query("best_match_score >= 0.80").sort_values(
    by=['id_left'], ascending=True)

In [341]:
matched_results_final[['id_left', 'id_right']].rename(
    columns={"id_left": "idACM", "id_right": "idDBLP"}).to_csv(
        'fuzzy_matching.csv', index=False)

### Example using Python Record Linkage Toolkit

In [236]:
acm_data = pd.read_csv(
    'https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/ACM.csv',
    index_col='id'
)
dblp_data = pd.read_csv(
    'https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/dblp.csv',
    index_col='id'
)


acm_data.dropna(subset=['authors'], inplace=True)
dblp_data.dropna(subset=['authors'], inplace=True)

acm_data['year'] = acm_data['year'].apply(str)
dblp_data['year'] = dblp_data['year'].apply(str)

In [237]:
# Build the indexer
indexer_full = recordlinkage.Index()
indexer_block = recordlinkage.Index()

indexer_full.full()
indexer_block.block(left_on='title', right_on='title')


<Index>

In [238]:
candidates = indexer_block.index(acm_data, dblp_data)

In [239]:
# Let's see how many matches we want to do
print(len(candidates))

984


In [240]:
print(acm_data.columns)
print(dblp_data.columns)

Index(['Unnamed: 0', 'title', 'authors', 'venue', 'year'], dtype='object')
Index(['Unnamed: 0', 'title', 'authors', 'venue', 'year'], dtype='object')


using Levenshtein comparison method as it works better with long strings.

[Source](https://srinivas-kulkarni.medium.com/jaro-winkler-vs-levenshtein-distance-2eab21832fd6)

In [241]:
# Takes 7 minutes using the full index.
# <1s using blocking
compare = recordlinkage.Compare()
compare.exact('year', 'year', label='year')
compare.string('authors',
               'authors',
               method='levenshtein',
               threshold=0.85,
               label='authors')
compare.string('title',
               'title',
               method='levenshtein',
               threshold=0.85,
               label='title')
compare.string('venue',
               'venue',
               method='levenshtein',
               threshold=0.85,
               label='venue')
features = compare.compute(candidates, acm_data,
                           dblp_data)

In [242]:
features.describe()

,year,authors,title,venue
count,984.000000,984.000000,984.0,984.0
mean,0.930894,0.321138,1.0,0.0
std,0.253763,0.467151,0.0,0.0
min,0.000000,0.000000,1.0,0.0
25%,1.000000,0.000000,1.0,0.0
50%,1.000000,0.000000,1.0,0.0
75%,1.000000,1.000000,1.0,0.0
max,1.000000,1.000000,1.0,0.0


In [243]:
# What are the score totals?
features.sum(axis=1).value_counts().sort_index(ascending=False)

3.0    304
2.0    624
1.0     56
dtype: int64

In [244]:
# Get the potential matches
potential_matches = features[features.sum(axis=1) > 1].reset_index()

In [245]:
print(potential_matches.shape)
potential_matches['Score'] = potential_matches.loc[:, 'authors':'authors'].sum(axis=1)
potential_matches.tail()

(928, 6)


,id_1,id_2,year,authors,title,venue,Score
923,672978,conf/vldb/JonscherD94,1,0.0,1.0,0.0,0.0
924,950483,journals/vldb/RamamurthyDS03,1,0.0,1.0,0.0,0.0
925,672977,conf/vldb/KemperK94,1,1.0,1.0,0.0,1.0
926,672980,conf/vldb/Guting94,1,0.0,1.0,0.0,0.0
927,672979,conf/vldb/WienerN94,1,0.0,1.0,0.0,0.0


In [246]:
acm_data.loc[950482,:]

Unnamed: 0                                                 2290
title                                           Guest editorial
authors       Philip A. Bernstein, Yannis Ioannidis, Raghu R...
venue         The VLDB Journal &mdash; The International Jou...
year                                                       2003
Name: 950482, dtype: object

In [247]:
dblp_data.loc['journals/vldb/BernsteinIR03',:]

Unnamed: 0                                                  671
title                                           Guest editorial
authors       Yannis E. Ioannidis, Raghu Ramakrishnan, Phili...
venue                                                   VLDB J.
year                                                       2003
Name: journals/vldb/BernsteinIR03, dtype: object

In [248]:
# Add some convenience columns for comparing data
acm_data['acm_authors_lookup'] = acm_data[[
    'year', 'authors', 'title', 'venue'
]].apply(lambda x: '_'.join(x), axis=1)

In [249]:
dblp_data['dblp_authors_lookup'] = dblp_data[[
     'year', 'authors', 'title', 'venue'
]].apply(lambda x: '_'.join(x), axis=1)

In [250]:
acm_data_lookup = acm_data[['acm_authors_lookup']].reset_index().rename(columns={"id": "id_1"})
dblp_data_lookup = dblp_data[['dblp_authors_lookup']].reset_index().rename(columns={"id": "id_2"})


In [251]:
acm_data_lookup.head()

,id_1,acm_authors_lookup
0,304586,"1999_Gottfried Vossen, Mathias Weske_The WASA2..."
1,304587,"1999_Isabel F. Cruz, Kimberly M. James_A user-..."
2,304589,"1999_Athman Bouguettaya, Boualem Benatallah, L..."
3,304590,"1999_Chaitan Baru, Amarnath Gupta, Bertram Lud..."
4,304582,"1999_Alexander Brodsky, Victor E. Segal, Jia C..."


In [252]:
dblp_data_lookup.head()

,id_2,dblp_authors_lookup
0,journals/sigmod/Mackay99,1999_D. Scott Mackay_Semantic Integration of E...
1,conf/vldb/PoosalaI96,"1996_Viswanath Poosala, Yannis E. Ioannidis_Es..."
2,conf/vldb/PalpanasSCP02,"2002_Themistoklis Palpanas, Richard Sidle, Ham..."
3,conf/vldb/GardarinGT96,"1996_Zhao-Hui Tang, Georges Gardarin, Jean-Rob..."
4,conf/vldb/HoelS95,"1995_Erik G. Hoel, Hanan Samet_Benchmarking Sp..."


In [253]:
acm_data_merge = potential_matches.merge(acm_data_lookup, how='left')

In [254]:
acm_data_merge.head()

,id_1,id_2,year,authors,title,venue,Score,acm_authors_lookup
0,277955,conf/sigmod/Epstein95,1,1.0,1.0,0.0,1.0,1995_Robert S. Epstein_Keynote Address_Interna...
1,277955,conf/sigmod/Ellison95,1,0.0,1.0,0.0,0.0,1995_Robert S. Epstein_Keynote Address_Interna...
2,276377,conf/sigmod/OlstonWACELSS98,1,0.0,1.0,0.0,0.0,"1998_Chris Olston, Allison Woodruff, Alexander..."
3,276367,conf/sigmod/Aulakh98,1,1.0,1.0,0.0,1.0,1998_Kamar Aulakh_About Quark Digital Media Sy...
4,304211,journals/vldb/MaratheS02,0,1.0,1.0,0.0,1.0,"1999_Arunprasad P. Marathe, Kenneth Salem_Quer..."


In [255]:
# Let's build a dataframe to  compare
final_merge = acm_data_merge.merge(dblp_data_lookup, how='left')

In [256]:
cols = [
    'id_1', 'id_2', 'Score', 'acm_authors_lookup',
    'dblp_authors_lookup'
]

In [257]:
final_merge[cols].sort_values(by=['id_1', 'Score'], ascending=False)

,id_1,id_2,Score,acm_authors_lookup,dblp_authors_lookup
795,959087,journals/sigmod/Hainaut03,1.0,2003_Jean-Luc Hainaut_Research in database eng...,2003_Jean-Luc Hainaut_Research in database eng...
794,959086,journals/sigmod/KambhampatiK03,1.0,"2003_Subbarao Kambhampati, Craig A. Knoblock_I...","2003_Subbarao Kambhampati, Craig A. Knoblock_I..."
797,959085,journals/sigmod/MotroA03,1.0,"2003_Amihai Motro, Troels Andreasen_Report on ...","2003_Amihai Motro, Troels Andreasen_Report on ..."
796,959084,journals/sigmod/Giorgini03,1.0,2003_Paolo Giorgini_Agent-Oriented software en...,2003_Paolo Giorgini_Agent-Oriented software en...
849,959083,journals/sigmod/LenzVJS03,0.0,"2003_Hans J. Lenz, Panos Vassiliadis, Manfred ...","2003_Manfred A. Jeusfeld, Hans-Joachim Lenz, P..."
...,...,...,...,...,...
745,234900,journals/sigmod/Dogac96,1.0,1996_Asuman Dogac_Middle East Technical Univer...,1996_Asuman Dogac_Middle East Technical Univer...
741,219774,journals/sigmod/Stacey95,1.0,"1995_Doug Stacey_Replication: DB2, Oracle, or ...","1995_Doug Stacey_Replication: DB2, Oracle, or ..."
740,212019,journals/sigmod/RisheSBDOALOSS95,0.0,"1995_Naphtali Rishe, Wei Sun, David Barton, Yi...","1995_Leonard Loureiro, Yi Deng, Carlos Ordonez..."
743,181569,journals/sigmod/Winslett94,1.0,1994_Marianne Winslett_Progress on HPCC and NI...,1994_Marianne Winslett_Progress on HPCC and NI...


In [258]:
final_merge[['id_1', 'id_2']].rename(columns={"id_1": "idACM", "id_2": "idDBLP"}).to_csv('record_linkage.csv',
                                                  index=False)

### Comparing to perfect mappings


In [342]:
perfect = pd.read_csv(
    'https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/LINKAGE/DATASETS/DBLP-ACM_perfectMapping.csv'
)
perfect = perfect[['idACM', 'idDBLP']].sort_values(by=['idACM']).reset_index()

fuzzy = pd.read_csv('fuzzy_matching.csv')
record = pd.read_csv('record_linkage.csv')

In [308]:
perfect

,index,idACM,idDBLP
0,1334,174639,journals/tods/SalemGS94
1,1331,174640,journals/tods/OliverS94
2,1332,174641,journals/tods/TendickM94
3,1333,174642,journals/tods/CliffordC94
4,1326,176568,journals/tods/RosenthalR94
...,...,...,...
2219,796,959083,journals/sigmod/LenzVJS03
2220,797,959084,journals/sigmod/Giorgini03
2221,798,959085,journals/sigmod/MotroA03
2222,799,959086,journals/sigmod/KambhampatiK03


In [309]:
record

,idACM,idDBLP
0,277955,conf/sigmod/Epstein95
1,277955,conf/sigmod/Ellison95
2,276377,conf/sigmod/OlstonWACELSS98
3,276367,conf/sigmod/Aulakh98
4,304211,journals/vldb/MaratheS02
...,...,...
923,672978,conf/vldb/JonscherD94
924,950483,journals/vldb/RamamurthyDS03
925,672977,conf/vldb/KemperK94
926,672980,conf/vldb/Guting94


In [343]:
fuzzy

,idACM,idDBLP
0,174639,journals/tods/SalemGS94
1,174640,journals/tods/OliverS94
2,174641,journals/tods/TendickM94
3,174642,journals/tods/CliffordC94
4,176568,journals/tods/RosenthalR94
...,...,...
2167,959083,journals/sigmod/LenzVJS03
2168,959084,journals/sigmod/Giorgini03
2169,959085,journals/sigmod/MotroA03
2170,959086,journals/sigmod/KambhampatiK03


In [344]:
f1_score(fuzzy, perfect)

NameError: ignored